In [1]:
import pandas as pd
import numpy as np
from scipy.io import arff
from sklearn.preprocessing import MinMaxScaler
import pickle
import torch
from models import *

In [2]:
def data_processing(data):
    data.dropna(inplace=True)
    data.reset_index(drop=True, inplace=True)
    categorical_features = list(data.select_dtypes(include='object').columns)
    categorical_features = list(set(categorical_features))
    numerical_features = [c for c in data.columns if c not in categorical_features]
    scaler = MinMaxScaler()
    data[numerical_features] = scaler.fit_transform(data[numerical_features])

    for _c in categorical_features:
        data[_c] = pd.Categorical(data[_c])
    df_transformed = pd.get_dummies(data, drop_first=True)
    return df_transformed, scaler


In [3]:
import numpy as np
dataset = 'loan_data_set.csv'
dataset = pd.read_csv(dataset).iloc[:, 1:]
dataset, scaler = data_processing(dataset)
y = np.array(dataset.iloc[:, -1], dtype=np.float32)
X = np.array(dataset.iloc[:, :-1], dtype=np.float32)
y = np.array(pd.get_dummies(y), dtype=np.float32)

In [7]:
from sklearn.metrics import pairwise_distances

def infy_pairwise_distance(a, b):
    return max(abs(a - b))

pairwise_distances(X, X, metric=infy_pairwise_distance).mean()

0.9730216706676198

In [18]:
folders = range(1, 6)
baseline = 'sl'
processed_baseline_results = []
for f in folders:
    results_baseline = pd.read_csv('{}/baseline_output_{}.csv'.format(f, baseline)).iloc[:, 1:]
    for i in range(results_baseline.shape[0]):
        processed_baseline_results.append([f, i + 1, results_baseline.iloc[i, :]['accuracy test corrected'], 
                                           results_baseline.iloc[i, :]['violations test'],
                                          results_baseline.iloc[i, :]['runtime']])

processed_baseline_results = pd.DataFrame(processed_baseline_results)
processed_baseline_results.columns = ['exp num', 'fold', 'test accuracy', 'test violations', 'runtime']
processed_baseline_results

,exp num,fold,test accuracy,test violations,runtime
0,1,1,0.861702,0.0,99.084469
1,1,2,0.761364,0.0,45.121334
2,1,3,0.804598,0.0,14.762682
3,1,4,0.776596,0.0,23.468920
4,1,5,0.804598,0.0,77.911794
5,2,1,0.811111,0.0,26.482244
6,2,2,0.771739,0.0,45.159947
7,2,3,0.846154,0.0,24.418242
8,2,4,0.752809,0.0,24.508349
9,2,5,0.833333,0.0,69.441050


In [19]:
print(np.mean(processed_baseline_results['test accuracy']), 
      np.std(processed_baseline_results['test accuracy']),
      np.mean(processed_baseline_results['test violations']), 
      np.std(processed_baseline_results['test violations']),
     np.mean(processed_baseline_results['runtime']), 
      np.std(processed_baseline_results['runtime']))

0.8017927690819502 0.03882445019543081 0.0 0.0 45.51626174926758 30.336309958722946


for each experiment, we read the models for each fold and aclculate adversarial robustness

In [22]:
def predict(model, X):
    X_torch = np.array(X)
    X_torch = X_torch.astype(np.float32)
    X_torch = torch.tensor(X_torch)
    
    score = model.forward(X_torch).detach().numpy()
    pred = []
    for p in score:
        pred.append(np.argmax(p))
    pred_d = np.zeros((len(pred), len(score[0])))
    pred_d[np.arange(len(pred)), pred] = 1
    return pred_d

from sklearn.metrics import accuracy_score
def calculate_corrected_accuracy(X, pred, y, K, corrected=True):
    if corrected:
        scaled_income = K.scale_[0] * (5000 - K.data_min_[0])
        indices_to_remove = np.where((X.T[0] >= 0) & (X.T[0] <= scaled_income) & 
                                     (X.T[4] == 0) & (y.T[1] == 1))[0]
        indices = list(set(range(len(X))) - set(indices_to_remove))
        return accuracy_score(pred[indices], y[indices])
    else:
        return accuracy_score(pred, y)

In [23]:
def get_adi(model, X, delta = 0.1):
    torch.onnx.export(model,                 # model being run
                  torch.tensor(X[:5]),       # model input (or a tuple for multiple inputs)
                  "baseline.onnx",           # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})
    onnx_model = onnx.load("baseline.onnx")
    onnx.checker.check_model(onnx_model)
    
    adi = 0
    for instance in X:
        network = Marabou.read_onnx(filename="baseline.onnx")
        inputVars = network.inputVars[0][0]
        outputVars = network.outputVars[0][0]
        scaled_income = scaler.scale_[0] * (5000 - scaler.data_min_[0])
        ranges = [[0, 1]]*len(inputVars)
        ranges[0] = [0, scaled_income]
        ranges[4] = [0, 0]
        instance[0] = scaled_income
        instance[4] = 0
        for i in range(len(inputVars)):
            network.setLowerBound(inputVars[i], ranges[i][0])
            network.setUpperBound(inputVars[i], ranges[i][1])
        
        for i in range(len(instance)):
            network.setLowerBound(inputVars[i], max(ranges[i][0], instance[i] - delta))
            network.setUpperBound(inputVars[i], min(ranges[i][1], instance[i] + delta))
        
        network.addInequality([13, 14], [1, -1], -0.1)
        options = Marabou.createOptions(verbosity = 2)
        vals = network.solve(options = options)
        if vals[0] == 'sat':
            adi += 1
    return adi/len(X)


In [24]:
import onnx
from maraboupy import Marabou

for e in folders:
    output = []
    for f in range(0, 5):
        model = torch.load(str(e) + '/baseline_model_fold_{}_{}.pt'.format(f + 1, baseline))
        pred = model.predict(X)
        for delta in [0.1, 0.01]:
            adi = get_adi(model, X, delta=delta)
            output.append([e, f + 1, delta, adi])
    output = pd.DataFrame(output)
    output.columns = ['experiment', 'fold', 'delta', 'adi']
    output.to_csv('{}/adi_{}.csv'.format(e, baseline))

unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsa

In [29]:
adi_results = pd.DataFrame()
for e in folders:
    adi = pd.read_csv(str(e) + '/adi_{}.csv'.format(baseline)).iloc[:, 1:]
    adi_results = adi_results.append(adi)

In [30]:
for delta in [0.1, 0.01]:
    print("adi for delta = {} is {} +/- {}".format(delta, 
                                                   np.mean(adi_results[adi_results.delta == delta].adi.tolist()), 
                                                   np.std(adi_results[adi_results.delta == delta].adi.tolist())))
    

adi for delta = 0.1 is 0.0018699186991869843 +/- 0.0041820556584102895
adi for delta = 0.01 is 0.00032520325203252 +/- 0.0011028178834350019


In [9]:
processed_deepsade_results = []
for f in folders:
    results_deepsade = pd.read_csv('{}/deepsade_output_ls.csv'.format(f)).iloc[:, 1:]
    mse_mean = np.mean(results_deepsade['accuracy test corrected'])
    violations_mean = np.mean(results_deepsade['violations test'])
    for i in range(results_deepsade.shape[0]):
        processed_deepsade_results.append([f, i + 1, results_deepsade.iloc[i, :]['accuracy test corrected'], 
                                           results_deepsade.iloc[i, :]['violations test'],
                                          results_deepsade.iloc[i, :]['runtime']])

processed_deepsade_results = pd.DataFrame(processed_deepsade_results)
processed_deepsade_results.columns = ['exp num', 'fold', 'test accuracy', 'test violations', 'runtime']
processed_deepsade_results

,exp num,fold,test accuracy,test violations,runtime
0,1,1,0.861702,0.0,478.601409
1,1,2,0.761364,0.0,481.699392
2,1,3,0.804598,0.0,669.372242
3,1,4,0.776596,0.0,292.809865
4,1,5,0.804598,0.0,446.416318
5,2,1,0.811111,0.0,602.036699
6,2,2,0.760870,0.0,453.823915
7,2,3,0.846154,0.0,331.661448
8,2,4,0.707865,0.0,349.881629
9,2,5,0.833333,0.0,550.702663


In [16]:
print(np.mean(processed_deepsade_results['test accuracy']), 
      np.std(processed_deepsade_results['test accuracy']),
      np.mean(processed_deepsade_results['test violations']), 
      np.std(processed_deepsade_results['test violations']),
     np.mean(processed_deepsade_results['runtime']), 
      np.std(processed_deepsade_results['runtime']))

0.800457476772035 0.042979282094931105 0.0 0.0 447.8835255813599 105.12229316006287
